In [1]:
%load_ext autoreload
%autoreload 2

In [33]:
import torch
import os
import pandas as pd
import torch.nn as nn
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split, Dataset
 
from tqdm import tqdm
from utils.london.process_data import build_train_test_loaders
from utils.london.training import CustomMAELoss, CustomMAPELoss, test_model
from utils.london.link_loads import get_graph_attributes, df_to_graph, build_quarter_hour_data, add_missing_nodes
from utils.london.models import STGCN
# import networkx as nx

In [36]:
MAE = CustomMAELoss()
MAPE = CustomMAPELoss()

In [37]:
folder_path = "data/london/"
# Get graph attributes, create dfs from csv, process Link column and ordered dfs by time
num_nodes, edge_index, node_mapping, dfs = get_graph_attributes(folder_path)

In [38]:
graph_data = []
# each df should have the same dimension and same nodes at the same columns
for filename, df in dfs.items():
    df = add_missing_nodes(df, node_mapping, num_nodes) # add zeros row for missing nodes
    df_qhrs = build_quarter_hour_data(df, filename, num_nodes) # retourne 24*4 df avec ses paramètres temporel et le flow
    graph_data.extend(df_qhrs)
    
graphs = [df_to_graph(df, edge_index) for df in graph_data]  # Un graphe par quart d'heure

## GCN + GRU (pas concluant, ne pas run)

In [46]:
class GraphSequenceDataset(Dataset):
    def __init__(self, graphs, window_size=4):
        self.graphs = graphs
        self.window_size = window_size
        if len(graphs) <= window_size:
            raise ValueError("Error : window_size should be higher than the number of graphs")
        
    def __len__(self):
        return len(self.graphs) - self.window_size

    def __getitem__(self, idx):
        # input_graphs prend en compte les window_size graphe précédent l'instant idx + le graphe à prèdire à l'instant idx
        input_graphs = self.graphs[idx : idx + self.window_size + 1]
        target = self.graphs[idx + self.window_size].y  # On prédit y du dernier graph
        return input_graphs, target 

In [47]:
# Définition des loaders
window_size = 4  # Nombre de pas de temps en entrée
train_size = int(0.8 * len(graphs))

train_dataset = GraphSequenceDataset(graphs[:train_size], window_size)  
test_dataset = GraphSequenceDataset(graphs[train_size:], window_size)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [48]:
class TGCN(nn.Module):
    def __init__(self, node_features, hidden_dim, gru_hidden_dim):
        super(TGCN, self).__init__()
        self.gcn = GCNConv(node_features, hidden_dim)  # GCN
        self.gru = nn.GRU(hidden_dim, gru_hidden_dim, batch_first=True)  # GRU
        self.fc = nn.Linear(gru_hidden_dim, 1)  # Prédiction finale

    def forward(self, graph_seq):
        # window_size = len(graph_seq)  # Nombre de pas de temps
        # batch_size = graph_seq[0].x.shape[0]  # Nombre de nœuds

        spatial_features = []
        for graph in graph_seq:
            x = self.gcn(graph.x, graph.edge_index)  # GCN
            x = F.relu(x)
            spatial_features.append(x)

        spatial_features = torch.stack(spatial_features, dim=1)  # (batch, time, hidden_dim)

        _ , final_state = self.gru(spatial_features)  # prédiction sur le dernier (1, gru_hidden_dim)
        final_state = final_state.squeeze() # (gru_hidden_dim)
        final_out = self.fc(final_state) # Prédiction sur le dernier état
        final_out = F.relu(final_out)
        return final_out

In [24]:
graphs[0]

Data(x=[1187, 10], edge_index=[2, 3946], y=[1187, 1])

In [50]:
model = TGCN(node_features=10, hidden_dim=32, gru_hidden_dim=64)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = MAE

# Boucle d'entraînement
for epoch in tqdm(range(50)):
    model.train()
    total_loss = 0
    for graph_seq, target in train_loader:
        optimizer.zero_grad()
        output = model(graph_seq)

        target = target.squeeze()
        output = output.reshape(target.shape)
    
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")


  2%|▏         | 1/50 [00:44<36:16, 44.42s/it]

Epoch 1, Loss: 256.43267001944605


  4%|▍         | 2/50 [01:29<35:38, 44.55s/it]

Epoch 2, Loss: 256.43267001944605


  4%|▍         | 2/50 [01:32<36:58, 46.21s/it]


KeyboardInterrupt: 

In [19]:
model.eval()
test_loss = 0
with torch.no_grad():
    for graph_seq, target in test_loader:

        output = model(graph_seq)
        loss = criterion(output, target)
        test_loss += loss.item()

print(f"Test MAE: {test_loss / len(test_loader)}")


Test MAE: 204.33262727453592


In [22]:
39e6/(96*1206)

336.8573797678275

The average flow per inter_station per 15min is 337. So 204 MAE error is not a valid result.

## GCN with past flows as node features

In [39]:
window_size=2
train_loader, test_loader = build_train_test_loaders(graphs, window_size)

In [42]:
from utils.london.models import STGCN

In [43]:
model = STGCN(node_features=window_size, hidden_dim=32)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = MAE

num_epochs = 50

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0
    for graph, temporal_features in train_loader:
        temporal_features = temporal_features.squeeze()
        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y    
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()        
        train_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            temporal_features = temporal_features.squeeze()  # Squeeze si nécessaire
            output = model(graph, temporal_features)  # Assure-toi de passer aussi les temporal_features
            target = graph.y
            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1, Train loss: 100.47351199780408


  2%|▏         | 1/50 [00:05<04:50,  5.93s/it]

Epoch 1, Test loss: 95.45939459920145
Epoch 2, Train loss: 96.26022369892029


  4%|▍         | 2/50 [00:12<04:50,  6.06s/it]

...

Epoch 48, Test loss: 93.34744163401244
Epoch 49, Train loss: 94.68297545829456


 98%|█████████▊| 49/50 [04:56<00:05,  5.90s/it]

Epoch 49, Test loss: 93.68424216027941
Epoch 50, Train loss: 94.66998190103735


100%|██████████| 50/50 [05:02<00:00,  6.06s/it]

Epoch 50, Test loss: 93.65443225195371


In [10]:
loss_mae = 0
loss_mape = 0
count_total = 0
for graph, temporal_features in test_loader:
    count = 0
    output = graph.x[:,-1]
    output = output.unsqueeze(-1)
    target = graph.y
    loss_mae += MAE(output, target)

    maskP = target >= 1
    countP = maskP.sum().item()

    outputP = output[maskP]
    targetP = target[maskP]

    if torch.all(graph.x[:,-1] == 0):
        print(graph.x)
        # print(targetP.shape)
    l = MAPE(outputP, targetP)
    loss_mape += l
    
    count_total += countP
    # print(count_total)

# print(count_total)
# print(loss_mape)

print("En utilisant la dernière valeur du flow comme prédiction :")
print(f"Test MAE: {loss_mae / len(test_loader)}")
print(f"Test MAPE: {loss_mape / count_total}")

tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [2.5924, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0157, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
  

Compliqué d'utiliser la MAPE car trop de targets égale ou proche de zéro